<a href="https://colab.research.google.com/github/bm919/CapstoneDesign/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import torch
from torchvision import models, transforms
from PIL import Image
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub


In [7]:
from getpass import getpass
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("🔐 Hugging Face API Key 입력: ")

🔐 Hugging Face API Key 입력: ··········


In [8]:
# CNN 모델 준비(예시)
cnn_model = models.resnet18(pretrained=True)
cnn_model.eval()

imagenet_labels = {
    504: "paper cup",  # 예시
    409: "can",
    620: "battery",
    # 필요한 라벨만 수동 추가 가능
}

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def predict_label(image_path):
    image = Image.open(image_path).convert("RGB")
    x = transform(image).unsqueeze(0)
    with torch.no_grad():
        logits = cnn_model(x)
    pred_id = logits.argmax(1).item()
    return imagenet_labels.get(pred_id, "unknown")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# CROMA DB 구축 예시
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docs = [
    "종이컵은 내용물을 비우고 물로 헹군 후 일반 종이류로 배출하세요.",
    "배터리는 폐건전지 수거함이나 동 주민센터에 따로 배출해야 합니다.",
    "캔은 내용물을 비우고 압착한 뒤 분리배출하세요."
]

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
documents = splitter.create_documents(docs)

if os.path.exists("./chroma_db"):
    import shutil
    shutil.rmtree("./chroma_db", ignore_errors=True)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="./chroma_db",
    collection_name="waste_info"
)

InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [11]:
# CROMA DB와 LLM 연동
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature": 0.2, "max_length": 256})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

In [12]:
# 사용자 이미지 입력 → 전체 파이프라인 실행
def full_pipeline(image_path):
    label = predict_label(image_path)
    print(f"[✅ 라벨 예측 결과] {label}")

    query = f"'{label}'은 어떻게 분리배출해야 하나요? 단계별로 쉽게 설명해줘."
    response = qa_chain.run(query)
    print("\n[🧠 LLM 응답]")
    print(response)

# 예시 실행
# full_pipeline("/content/sample_image.jpg")